In [ ]:
!pip install accelerate


In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model_name='/home/ubuntu/Mistral-7B-Instruct-v0.2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    quantization_config = bnb_config
    )
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer = tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto", max_new_tokens=5
)
hf = HuggingFacePipeline(pipeline=pipe)

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
import os
prompt_template = ''' <s> [INST] You are an analytical tool specialized in processing and classifying GitHub Commit message. Your task is to assess developer's intent in a given commit message and categorize it into one of the following predefined categories based on its content:
                      
                      'Yes':  A commit messages that explicitly mentions performance improvement or optimization, specifically in terms of execution time or resource utilization. The message should clearly indicate actions that made the code runs faster or  more efficiently, use less memory, or more efficiently utilize system resources. Also, if a commit message describes a change made to address a performance bottleneck, prevent performance degradation, reduce overheads or solve a problem that negatively affects performance. This includes optimizations like replacing inefficient code patterns that are known to kill performance even if the message does not use the words 'improvement' or 'performance' explicitly.
                      'No': A commit message that do not pertain to performance enhancements. This includes messages related to code changes for testing, documentation, performance profiling/monitoring/debugging/analysis and bug/error/crash fixes that don't explicitly mention performance improvement of the application itself, code refactoring or feature addition without explicit performance optimization,  and mentions of necessary or speculative or potential performance enhancements without concrete evidence or results. Also, a messages that is irrelevant, unclear, or ambiguous, and those that do not provide enough context to determine their intent.     

                    If the commit message doesn't fit clearly into any of the above categories, classify it as: 'No'. Additionally, pay close attention to the context in which terms like 'optimize', 'performance', 'improve' or 'improvements' are used. Not all improvements are related to performance—only, classify a message as 'Yes' if it specifically mentions enhancements related to execution time, memory usage, or resource efficiency. Avoid making assumptions based on ambiguous terms. You should have high confidence in classifying a message as 'Yes' based on careful examination of the information provided in the commit message.
                    If you encounter a commit message with multiple intentions, where at least one of those intentions includes a performance improvement, classify the entire message as 'Yes'.
                    You will only respond with the predefined category. Do not include the word 'Category'. Do not provide explanations or notes.
                    
                    Commit message : ```{commit_message}``` [/INST] Model_answer:  </s> '''



prompt = PromptTemplate(
    input_variables=["commit_message"], template=prompt_template
)
llm = LLMChain(llm=hf, prompt=prompt)

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Specify the path to your JSON file
json_file_path = '/home/ubuntu/data/train.jsonl'
file_test = '/home/ubuntu/data/test.jsonl'
# Read the JSON file into a Pandas DataFrame
df_train = pd.read_json(json_file_path, lines = True)

df_test = pd.read_json(file_test,lines=True)
print(df_train)

In [ ]:


# Assuming df_train is already loaded and contains a target column named 'target'

# Define the range you're interested in
start_index = 1
end_index = 2180

# Filter df_train for the specified range before separating it based on the 'target' column
df_train_filtered = df_train.loc[start_index:end_index]

# Separate the filtered DataFrame into two based on the target column values
df_0 = df_train_filtered[df_train_filtered['target'] == 0]
df_1 = df_train_filtered[df_train_filtered['target'] == 1]

# Sample rows from each subset
# Note: Ensure there are at least 1000 rows in each filtered subset to avoid errors
min_count = min(len(df_0), len(df_1))
sample_0 = df_0.sample(n=min_count, random_state=42)
sample_1 = df_1.sample(n=min_count, random_state=42)

# Concatenate the two samples to get a balanced dataset
df_balanced_sample = pd.concat([sample_0, sample_1])

print(df_balanced_sample)




# # Assuming df_test is already loaded and contains a target column named 'target'

# # Separate the DataFrame into two based on the target column values
# df_0 = df_train[df_train['target'] == 0]
# df_1 = df_train[df_train['target'] == 1]

# # Calculate the minimum count to ensure equal number of 1s and 0s
# #min_count = min(len(df_0), len(df_1))

# # Sample min_count rows from each subset
# sample_0 = df_0.sample(n=1000, random_state=42)
# sample_1 = df_1.sample(n=1000, random_state=42)

# # Concatenate the two samples to get a balanced dataset
# df_balanced_sample = pd.concat([sample_0, sample_1])

# print(df_balanced_sample)


Now do predict with the llm

In [ ]:
Predictions=[]
for i in df_balanced_sample['commit_message']:
    Predictions.append(llm.run(i))

In [ ]:
import re
df_balanced_sample['mistral_result']=Predictions
df_balanced_sample['mistral_result_cleaned']=df_balanced_sample['mistral_result'].apply(lambda x: re.search(r'\b(Yes|No)\b', x).group(0) if re.search(r'\b(Yes|No)\b', x) else None)

In [ ]:
print(df_balanced_sample)

In [ ]:
df_balanced_sample['mistral_result_cleaned'].unique()

In [ ]:
df_balanced_sample['mistral_target']=df_balanced_sample['mistral_result_cleaned'].replace('Yes',1).replace('No',0)

In [ ]:
df_balanced_sample['mistral_target']

In [ ]:
df_balanced_sample['mistral_target'].value_counts()

In [ ]:
df_balanced_sample['mistral_target']=df_balanced_sample['mistral_target'].fillna(0)

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.metrics import classification_report

#y_true= df_balanced_sample['target']
#y_pred=df_balanced_sample['mistral_target']
y_true= df_balanced_sample[df_balanced_sample['mistral_target'].notnull()]['target']
y_pred=df_balanced_sample[df_balanced_sample['mistral_target'].notnull()]['mistral_target']

# Generate the classification report
report = classification_report(y_true, y_pred,output_dict=True)

# Print the classification report
print(report)


In [ ]:
y_true.value_counts()


In [ ]:
y_pred.value_counts()

In [ ]:
!pip install seaborn

In [ ]:
!pip install matplotlib

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
df1 = pd.DataFrame(report).transpose()
df1 = df1.sort_values(by=['f1-score'], ascending=False)

# Plot the classification report
plt.figure(figsize=(10, 5))
sns.heatmap(df1[['precision', 'recall', 'f1-score']], annot=True, cmap='plasma')
plt.title('Classification Report')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)

# Extract true positives (TP), false positives (FP), false negatives (FN), and true negatives (TN)
tn, fp, fn, tp = cm.ravel()

# Calculate false positives and false negatives
false_positive = fp
false_negative = fn

print("False Positives:", false_positive)
print("False Negatives:", false_negative)
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Positive', 'Predicted Negative'], yticklabels=['Actual Positive', 'Actual Negative'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Assuming y_true and y_pred are already defined as shown previously
import numpy as np
unique_labels = np.unique(np.concatenate((y_true, y_pred)))

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=unique_labels)

# Plot the confusion matrix as a heatmap with unique class labels
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
            xticklabels=unique_labels, 
            yticklabels=unique_labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

We need to know the false positives and false negetives for tuning the model

In [ ]:
pos = df_balanced_sample[ df_balanced_sample['mistral_target'] == 1]
pos

In [ ]:
fp = pos[pos['target'] == 0]
fp.to_csv('false_positive.csv', index=False)

In [ ]:
pos = df_balanced_sample[ df_balanced_sample['mistral_target'] == 0]  #for false negetives
pos

In [ ]:
fn = pos[pos['target'] == 1]
fn.to_csv('false_negetive.csv', index=False)

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# import torch
# # Specify the device for CUDA acceleration
# device = "cuda" # Or "cpu" if you're not using NVIDIA GPUs
# # Configuration for BitsAndBytes to optimize model loading
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True, # Enables loading the model in 4-bit precision
#     bnb_4bit_quant_type="nf4", # Specifies the quantization type
#     bnb_4bit_use_double_quant=True, # Enables double quantization for better precision
# )
# # Loading the tokenizer
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# # Loading the model with BitsAndBytes configuration, and additional settings from Method-1
# model = AutoModelForCausalLM.from_pretrained(
#     "mistralai/Mistral-7B-Instruct-v0.2",
#     torch_dtype=torch.float16, # Sets the tensor type to float16 for faster computation
#     device_map="auto", # Automatically maps the model layers to the available devices
#     trust_remote_code=True, # Allows the execution of remote code for custom model configurations
#     attn_implementation="flash_attention_2", # Uses a specific attention implementation optimized for performance
#     _config=bnb_config, # Applies the BitsAndBytes configuration
# )
# # Move the model to the specified device
# model.to(device)
# # Prepare the messages for encoding
# messages = [
#     {"role": "user", "content": "What is your favourite condiment?"},
#     {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
#     {"role": "user", "content": "Do you have mayonnaise recipes?"}
# ]
# # Encode the messages using the tokenizer
# encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
# # Generate responses using the model
# generated_ids = model.generate(encodeds, max_new_tokens=5, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# # Print the generated response
# print(decoded[0])